In [1]:
from pathlib import Path
import os

STUDENT = 'mmr497'
DATA_PATH = Path('/local/data/mmr497')
OUTLIERS_PATH = Path('./outliers2/')
INTERMEDIATE_PATH = Path(f'{DATA_PATH}/intermediate_datafiles/')
os.chdir(f'/home/{STUDENT}/')
from util.VisualizeDataset import VisualizeDataset
from Visualiser import Visualiser as Viz
from util.util import ignore_actual_time, read_parquet, write_parquet
from FeatureCreator import FeatureCreatorUpdated
from DataLearningLoader import DataLearningLoader
import pandas as pd
from Chapter7.FeatureSelection import FeatureSelectionClassification
from Chapter7.LearningAlgorithms import ClassificationAlgorithms
from AlteredAlgorithms import AlteredAlgorithmsClassifier
from Chapter7.Evaluation import ClassificationEvaluation
import numpy as np
from CustomPipeline import PreConfiguredPipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Testing parameters
feature_selection = True

In [60]:
EasyViz = Viz()
DataViz = VisualizeDataset('ML2.ipynb')

In [4]:
# cleaned_dataset = read_parquet(INTERMEDIATE_PATH / 'ML4QS_imputed_results.parquet')

In [5]:
# instance_mask = cleaned_dataset.id == 1
# col = 'location_phone_Velocity'
# print(cleaned_dataset.loc[instance_mask, col].isna().sum() / len(cleaned_dataset.loc[instance_mask, col]))
# if cleaned_dataset.loc[instance_mask, col].isna().all():
#     print('All values are NaN for instance {}.'.format(cleaned_dataset.loc[instance_mask, 'id'].values[0]))

In [6]:
# cleaned_dataset.dtypes

In [7]:
# test_df = read_parquet(INTERMEDIATE_PATH / 'non_fourier_features.parquet')
# nan_counts = test_df.isna().sum()
# nan_cols = nan_counts[nan_counts > 0]
# print(nan_cols)

In [8]:
# fcr = FeatureCreatorUpdated(INTERMEDIATE_PATH)
# DLL = DataLearningLoader(df_path=INTERMEDIATE_PATH, output_dir=INTERMEDIATE_PATH, verbose=False)

In [9]:
# feature_df = fcr.create_features(cleaned_dataset, overwrite=True)

In [10]:
# nan_counts = feature_df.isna().sum()
# nan_cols = nan_counts[nan_counts > 0]
# print(nan_cols)

In [11]:
# # Stupid fix! Only temporary! MUST REMOVE
# def remove_nan_cols(df):
#     df_fs = df.copy()
#     col_before = df_fs.columns
#     nan_counts = df_fs.isna().sum()
#     nan_cols = nan_counts[nan_counts > 0]
#     df_fs = df_fs.drop(nan_cols.index, axis=1)
#     col_after = df_fs.columns
#     print('Removed columns: ', [col for col in col_before if col not in col_after])
#     return df_fs
#
# filtered_feature_df = remove_nan_cols(feature_df)

In [12]:
# X_train, X_test, y_train, y_test = DLL.prepare_data(filtered_feature_df)

In [53]:
intermediate_df = read_parquet(INTERMEDIATE_PATH / 'ML4QS_combined_results_final.parquet')

In [54]:
intermediate_df.dtypes

id                                      int64
acc_phone_X                           float64
acc_phone_Y                           float64
acc_phone_Z                           float64
lin_acc_phone_X                       float64
lin_acc_phone_Y                       float64
lin_acc_phone_Z                       float64
gyr_phone_X                           float64
gyr_phone_Y                           float64
gyr_phone_Z                           float64
location_phone_Latitude               float64
location_phone_Longitude              float64
location_phone_Height                 float64
location_phone_Velocity               float64
location_phone_Direction              float64
location_phone_Horizontal Accuracy    float64
location_phone_Vertical Accuracy      float64
mag_phone_X                           float64
mag_phone_Y                           float64
mag_phone_Z                           float64
proximity_phone_Distance              float64
labeltram                         

In [55]:
len(intermediate_df)

103624

In [63]:
intermediate_df = ignore_actual_time(intermediate_df)

In [64]:
EasyViz.plot_dataset(intermediate_df, plot_type='full', path='plot_dataset_final')

Figure saved to figures/plot_dataset_final/figure_1.png
Figure saved to figures/plot_dataset_final/figure_1.pdf


In [5]:
def write_splitted_data(data):

    if len(data[0]) == 1:
        print("Expecting format: (data, name) ... continuing.")

    for data, name in data:
        try:
            write_parquet(data, INTERMEDIATE_PATH / f'{name}_postpipe.parquet')
        except:
            print("Can't write parquet file for ", name, " ... continuing.")

In [ ]:
# pipe_NF = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH, include_fourier=False)
# X_train, X_test, y_train, y_test = pipe_NF.fit_transform(intermediate_df, verbose=True, overwrite=True, pipe_name="NF", srtkfold=False)

In [ ]:
# nf_data = [(X_train, "X_train_NF"), (X_test, "X_test_NF"), (y_train, "y_train_NF"), (y_test, "y_test_NF")]
# write_splitted_data(nf_data)

In [ ]:
# pipe = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH)
# X_train, X_test, y_train, y_test = pipe.fit_transform(intermediate_df, verbose=True, overwrite=False, srtkfold=False)

In [ ]:
pipe = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH)
split_sets = pipe.fit_transform(intermediate_df, verbose=True, overwrite=True, srtkfold=True)

In [11]:
split = 1
for X_train, X_test, y_train, y_test in split_sets:
    split_data = [(X_train, f"X_train_KF{split}"), (X_test, f"X_test_KF{split}"), (y_train, f"y_train_KF{split}"), (y_test, f"y_test_KF{split}")]
    write_splitted_data(split_data)
    split +=1

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/X_train_KF1_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/X_test_KF1_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/y_train_KF1_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/y_test_KF1_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/X_train_KF2_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/X_test_KF2_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/y_train_KF2_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/y_test_KF2_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/X_train_K

In [ ]:
# pipe = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH)
# X_train, X_test, y_train, y_test = pipe.fit_transform(intermediate_df, verbose=True, overwrite=False)

In [18]:
# y_test

timestamp
1970-01-01 00:54:19.750    metro
1970-01-01 00:54:20.000    metro
1970-01-01 00:54:20.250    metro
1970-01-01 00:54:20.500    metro
1970-01-01 00:54:20.750    metro
                           ...  
1970-01-01 03:30:08.000    train
1970-01-01 03:30:08.250    train
1970-01-01 03:30:08.500    train
1970-01-01 03:30:08.750    train
1970-01-01 03:30:09.000    train
Name: transport_mode, Length: 20479, dtype: object

In [ ]:
# write_splitted_data([(X_train, "X_train"), (X_test, "X_test"), (y_train, "y_train"), (y_test, "y_test")])

In [52]:
def read_splitted_data(splits=3):
    splitted_data = []
    for i in range(1, splits+1):
        X_train = read_parquet(INTERMEDIATE_PATH / f'X_train_KF{i}_postpipe.parquet')
        X_test = read_parquet(INTERMEDIATE_PATH / f'X_test_KF{i}_postpipe.parquet')
        y_train = read_parquet(INTERMEDIATE_PATH / f'y_train_KF{i}_postpipe.parquet')
        y_test = read_parquet(INTERMEDIATE_PATH / f'y_test_KF{i}_postpipe.parquet')
        splitted_data.append([X_train, X_test, y_train, y_test])
    return splitted_data

test_splitted_data = read_splitted_data(3)

In [4]:
# Optional for testing

def remove_problem_cols(df):
    # This is for testing purposes, stupid fix!
    df_fs = df.copy()
    col_before = df_fs.columns
    df_fs = df_fs.select_dtypes(include=[np.number], exclude=['timedelta64[ns]'])
    col_after = df_fs.columns
    print('Removed columns: ', [col for col in col_before if col not in col_after])
    return df_fs

def feature_selection(X_train, X_test, y_train, y_test):
    fs = FeatureSelectionClassification()

    N_FORWARD_SELECTION = 50

    X_train_fs = remove_problem_cols(X_train)
    X_test_fs = remove_problem_cols(X_test)

    features, ordered_features, ordered_scores = fs.forward_selection(N_FORWARD_SELECTION,
                                                                          X_train_fs,
                                                                          X_test_fs,
                                                                          y_train,
                                                                          y_test,
                                                                          gridsearch=True)

    return features, ordered_features, ordered_scores

        # DataViz.plot_xy(x=[range(1, N_FORWARD_SELECTION+1)], y=[ordered_scores],
        #             xlabel='number of features', ylabel='accuracy')

In [20]:
class EvaluationContainer:
    def __init__(self, model_name):
        self.model_name = model_name

In [26]:
# nan_counts = X_test_fs.isna().sum()
# nan_cols = nan_counts[nan_counts > 0]
# print(nan_cols)

location_phone_Velocity           4522
location_phone_Velocity_mean      4522
location_phone_Velocity_median    4522
location_phone_Velocity_min       4522
location_phone_Velocity_max       4522
session_avg_velocity              4522
session_velocity_std              4522
dtype: int64


In [10]:
def plot_class_distribution(data, title: str = 'Class distribution'):
    if isinstance(data, pd.Series):
        data = pd.Series(data)

    value_counts = data['transport_mode'].fillna('missing').astype(str).value_counts()

    plt.figure(figsize=(10, 6))
    value_counts.plot(kind='bar')
    plt.title(title)
    plt.xlabel('Transport Mode')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    plt.savefig(f'/local/data/mmr497/figures/class_distribution.png')
    plt.close()

In [58]:
train_labels = [df[2] for df in test_splitted_data]
test_labels = [df[3] for df in test_splitted_data]
train_dfs = pd.concat(train_labels, ignore_index=True)
test_dfs = pd.concat(test_labels, ignore_index=True)

all_labels = pd.concat([train_dfs, test_dfs], ignore_index=True)

plot_class_distribution(all_labels, 'class distribution in total')

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [57]:
print(len(all_labels))

302778


In [39]:
# y_test_test['transport_mode'].dtypes

dtype('O')

In [ ]:
# plot_class_distribution(y_test_test)

In [50]:
# Setup model
CA = ClassificationAlgorithms()
alt_CA = AlteredAlgorithmsClassifier()

y_test_combined = []

pred_y_train_results = []
pred_prob_y_train_results = []
pred_y_test_results = []
pred_prob_y_test_results = []

for X_train, X_test, y_train, y_test in test_splitted_data:
    X_train_fs = remove_problem_cols(X_train)
    X_test_fs = remove_problem_cols(X_test)
    y_test_combined.extend(y_test.values)

    nb_train_y, nb_test_y, nb_train_prob_y, nb_test_prob_y = alt_CA.naive_bayes(
            X_train_fs, y_train, X_test_fs
        )
    pred_y_train_results.extend(nb_train_y)
    pred_prob_y_train_results.extend(nb_test_prob_y)
    pred_y_test_results.extend(nb_test_y)
    pred_prob_y_test_results.extend(nb_test_prob_y)

Removed columns:  ['original_time', 'time_diff', 'shifted_time']
Removed columns:  ['original_time', 'time_diff', 'shifted_time']
Removed columns:  ['original_time', 'time_diff', 'shifted_time']
Removed columns:  ['original_time', 'time_diff', 'shifted_time']
Removed columns:  ['original_time', 'time_diff', 'shifted_time']
Removed columns:  ['original_time', 'time_diff', 'shifted_time']


In [42]:
test_test = []

for X_train, X_test, y_train, y_test in test_splitted_data:
    test_test.extend(y_test.values)

print(len(test_test))

100926


In [47]:
print(y_test_combined)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [51]:
# Evaluate models
evalCA = ClassificationEvaluation()

evaluations = {}

print(classification_report(y_test_combined, pred_y_test_results, zero_division=0))
print(confusion_matrix(y_test_combined, pred_y_test_results))

              precision    recall  f1-score   support

        bike       1.00      1.00      1.00     17558
         bus       0.00      0.00      0.00      8430
         car       0.38      0.17      0.23     14415
       metro       0.00      0.00      0.00      6318
       train       0.56      1.00      0.72     29927
     walking       1.00      1.00      1.00     24278

    accuracy                           0.73    100926
   macro avg       0.49      0.53      0.49    100926
weighted avg       0.64      0.73      0.66    100926

[[17550     0     0     0     0     8]
 [    1     0  3757     0  4672     0]
 [    0     0  2425     0 11990     0]
 [    0     0     0     0  6318     0]
 [    0     0   123     1 29803     0]
 [    0     0     0     0     0 24278]]
